# Bias and RMSE Calculations Demo

In [1]:
# imports
import sys
!{sys.executable} -m pip install xarray
import os
import fnmatch
import numpy as np
import pandas as pd
import xarray as xr
cwd = os.getcwd()
cwd

ImportError: libtbb.so.2: cannot open shared object file: No such file or directory

In [ ]:
# get files
for file_name in os.listdir('/home/eudoraf12/Livneh Wrfcf NG-GR-NQ-BD/script_demo'):
    if fnmatch.fnmatch(file_name, 'wrf_2017020500_NG-GR-NQ-BD_'+'max'+'*_prev.nc'):
        print(file_name)

In [ ]:
# helper variables
fn = '/cw3e/mead/projects/cwp103/scratch/ldehaan/Livneh/ptt_2017.nc'
liv_file = xr.open_dataset(fn)
liv_days = [36,37,38,39,40,41,42,43,44,45]    # AR case specific
days_next = ['n0','n1','n2','n3','n4','n5','n6','n7','n8','n9']
days_prev = ['p0','p1','p2','p3','p4','p5','p6','p7','p8','p9']

# class for calculating bias and RMSE
class stats:
    
    # constructor initializing case, run, and tmax/tmin option
    def __init__ (self, case, run, maxmin):
                
        # input verification
        if isinstance(case, str) == False:
            raise Exception('Case definition is not a string.')
        elif isinstance(run, str) == False:
            raise Exception('Run definition is not a string.')
        elif maxmin not in ['max','min']:
            raise Exception('Max or Min was not defined.')
        
        # attribute constructor
        self.case = case
        self.run = run
        self.maxmin = maxmin
        self.fn_prev = []
        self.fn_next = []
        self.base_df = pd.DataFrame()
        
    # populating initial dataframe
    def insert_col(self, file_lst, df, timing_str):
        num = 0
        for x in sorted(file_lst):
            dt = (xr.open_dataarray('./' + x)).data.flatten()
            print(x)
            df[timing_str + str(num)] = dt
            num += 1
        
    # collecting filenames for next and prev timing options, populating base_df
    def read(self):
        for file_name in os.listdir('/home/eudoraf12/Livneh Wrfcf NG-GR-NQ-BD/script_demo'):
            if fnmatch.fnmatch(file_name, 'wrf_' + self.case + '_' + self.run + '_' + self.maxmin + '*_prev.nc'):
                self.fn_prev.append(file_name)
            elif fnmatch.fnmatch(file_name, 'wrf_' + self.case + '_' + self.run + '_' + self.maxmin + '*_next.nc'):
                self.fn_next.append(file_name)
        self.insert_col(self.fn_prev, self.base_df, 'p')
        self.insert_col(self.fn_next, self.base_df, 'n')
        
    # df getter method
    def df_getter(self):
        return self.base_df
    
    # fn_prev getter method
    def fn_prev_get(self):
        return sorted(self.fn_prev)
    
    # subtracting dataframe col from livneh
    def diff(self, num_str, day, table, out, maxmin_str):
        liv = liv_file[maxmin_str][day].data.flatten()
        table[num_str][table[num_str] == 0.0] = np.NAN
        table[num_str] = table[num_str] - 273.15
        subtr = np.subtract(np.array(table[num_str]), liv)
        out[num_str] = subtr
    
    # calculating bias
    def bias_calc(self, diff_df):
        bias = diff_df.values.flatten()[~np.isnan(diff_df.values.flatten())].mean()
        return 'Bias: ' + str(round(bias, 2))
    
    # calculating RMSE
    def rmse_calc(self, diff_df):
        sq_all = diff_df**2
        rmse = np.sqrt(sq_all.values.flatten()[~np.isnan(sq_all.values.flatten())].mean())
        return 'RMSE: ' + str(round(rmse, 2))
    
    # bias and RMSE calculation for next-day and previous-day timing options
    def next_prev_calc(self):
        next_pd = pd.DataFrame()
        prev_pd = pd.DataFrame()
        for a in np.arange(len(liv_days)):
            self.diff(days_next[a], liv_days[a], self.base_df, next_pd, 'T' + self.maxmin)
            self.diff(days_prev[a], liv_days[a], self.base_df, prev_pd, 'T' + self.maxmin)
        return self.bias_calc(next_pd), self.rmse_calc(next_pd), self.bias_calc(prev_pd), self.rmse_calc(prev_pd)

In [ ]:
test = stats('2017020500','NG-GR-NQ-BD','max')

In [ ]:
test.read()
datafrm = test.df_getter()
print(datafrm)
print(test.fn_prev_get())
#test.next_prev_calc()

In [ ]:
lst = ['NG-GR-YQ-BD','NG-SP-NQ-PO','OO-6D','OO-CO','OO-SC','PH-MO-GF-MY','PH-MO-TK-MY',
       'PH-TH-GF-MY','PH-TH-TK-MY','VL-100','VL-120','NG-GR-NQ-PO','NG-GR-YQ-PO','NG-SP-YQ-PO','OO-CC']

In [ ]:
#for i in lst:
    #temp = stats('2017020500',i,'max')
    #temp.read()
    #print(temp.next_prev_calc())

In [ ]:
# verifying results using know correct method
out_prev = pd.DataFrame()
out_next = pd.DataFrame()
def diff(num_str, day, table, out):
    fn = '/cw3e/mead/projects/cwp103/scratch/ldehaan/Livneh/ptt_2017.nc'
    liv_file = xr.open_dataset(fn)
    liv = liv_file['Tmax'][day].data.flatten()
    table[num_str][table[num_str] == 0.0] = np.NAN
    table[num_str] = table[num_str] - 273.15
    subtr = np.subtract(np.array(table[num_str]), liv)
    out[num_str] = subtr

liv_days = [36,37,38,39,40,41,42,43,44,45]
for i in np.arange(len(liv_days)):
    diff(days_prev[i],liv_days[i],datafrm,out_prev)
    diff(days_next[i],liv_days[i],datafrm,out_next)
bias_prev = out_prev.values.flatten()[~np.isnan(out_prev.values.flatten())].mean()
bias_next = out_next.values.flatten()[~np.isnan(out_next.values.flatten())].mean()
sq_all_prev = out_prev**2
rmse_prev = np.sqrt(sq_all_prev.values.flatten()[~np.isnan(sq_all_prev.values.flatten())].mean())
sq_all_next = out_next**2
rmse_next = np.sqrt(sq_all_next.values.flatten()[~np.isnan(sq_all_next.values.flatten())].mean())
print('Prev Bias, Prev Rmse')
print(round(bias_prev,2), round(rmse_prev,2))
print('Next Bias, Next Rmse')
print(round(bias_next,2), round(rmse_next,2))

notes:
8/24/21
- know that the files are correct, since they gave us the same stats through previous calc method
- need to fix the way the dataframes are being populated